# Food Derserts and Birth Outcomes | ML for cities final project

- controlling for zip code specific factors (controlling for everything else, does being in a food desert matter to the outcome of low birth weight?) 
- pick one definition of a food desert OR take some of the variables that make up whether it is a fd and put that in the predictor variables 
- number for feature (sklearn variable importance) 
- need to use a different ml technique to really get interpretability / variable importance 
- compare accuracy of different models
- bayes net vs random forests 
- equal frequency bins (quartiles/quintiles) 

## Merging data before analysis

In [66]:
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pylab as plt
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

## Read in the cleaned ACS data set

In [377]:
dfACS = pd.read_csv('dataACS.csv')
# dfACS = dfACS.set_index('zipcode')
# dfACS = dfACS.drop('Unnamed: 0', axis=1)
dfACS.zipcode = dfACS.zipcode.astype(str)
dfACS.head()

,zipcode,prctForeign,%Uninsured,%vehicle,%foodStamp,%poverty,urban/rural
0,10001,16.230927,6.313218,77.510070,51.290074,17.5,1
1,10002,16.301731,7.873643,76.876850,42.674401,27.8,1
2,10003,12.463227,5.066324,77.210118,48.017334,9.9,1
3,10004,15.177398,6.241787,76.513672,52.299606,4.8,1
4,10005,16.211251,5.774971,85.056784,49.311137,12.1,1


In [378]:
dfACS.index.shape

(5212,)

In [379]:
dfACS.dtypes

zipcode         object
prctForeign    float64
%Uninsured     float64
%vehicle       float64
%foodStamp     float64
%poverty       float64
urban/rural      int64
dtype: object

## Read in Birth outcomes data set

In [397]:
birth = pd.read_csv('birthNYS.csv')
birth = birth.drop(['Unnamed: 0'], axis = 1)
# birth = birth.set_index('ZipCode')
birth.head(3)

,TeenBirthRate,ZipCode,TotalBirths2012-2014,PrematureBirth,LowBirthWeight
0,5.6,14008,34.0,2.9,0.0
1,13.6,14012,67.0,10.4,6.0
2,5.1,14028,47.0,8.5,10.6


In [598]:
df = dfACS.merge(birth, left_on = 'zipcode', right_on = 'ZipCode')
df.head()

,zipcode,prctForeign,%Uninsured,%vehicle,%foodStamp,%poverty,urban/rural,TeenBirthRate,ZipCode,TotalBirths2012-2014,PrematureBirth,LowBirthWeight
0,10001,16.230927,6.313218,77.510070,51.290074,17.5,1,5.7,10001,527.0,10.5,11.0
1,10002,16.301731,7.873643,76.876850,42.674401,27.8,1,17.7,10002,3044.0,8.9,6.6
2,10003,12.463227,5.066324,77.210118,48.017334,9.9,1,0.6,10003,1268.0,9.7,8.7
3,10004,15.177398,6.241787,76.513672,52.299606,4.8,1,0.0,10004,195.0,8.7,6.7
4,10005,16.211251,5.774971,85.056784,49.311137,12.1,1,0.0,10005,364.0,10.7,6.0


In [599]:
print(df.shape)
df.columns

(2059, 12)


Index(['zipcode', 'prctForeign', '%Uninsured', '%vehicle', '%foodStamp',
       '%poverty', 'urban/rural', 'TeenBirthRate', 'ZipCode',
       'TotalBirths2012-2014', 'PrematureBirth', 'LowBirthWeight'],
      dtype='object')

In [600]:
df = df.drop(['ZipCode', 'TotalBirths2012-2014'], axis=1)

print(df.shape)
df.columns

(2059, 10)


Index(['zipcode', 'prctForeign', '%Uninsured', '%vehicle', '%foodStamp',
       '%poverty', 'urban/rural', 'TeenBirthRate', 'PrematureBirth',
       'LowBirthWeight'],
      dtype='object')

In [601]:
print(len(df.zipcode))
print(len(df.zipcode.unique()))

2059
1454


In [602]:
df = df.drop_duplicates(subset=['zipcode'])
print(df.shape)

(1454, 10)


In [603]:
# df = df.set_index('zipcode')
df.columns = ['zipcode', '%Foreign', '%Uninsured', '%vehicle', '%foodStamp', '%poverty', 'urban', 
              '%TeenBirthRate', '%PrematureBirth', '%LowBirthWeight']
df.head()

,zipcode,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%TeenBirthRate,%PrematureBirth,%LowBirthWeight
0,10001,16.230927,6.313218,77.510070,51.290074,17.5,1,5.7,10.5,11.0
1,10002,16.301731,7.873643,76.876850,42.674401,27.8,1,17.7,8.9,6.6
2,10003,12.463227,5.066324,77.210118,48.017334,9.9,1,0.6,9.7,8.7
3,10004,15.177398,6.241787,76.513672,52.299606,4.8,1,0.0,8.7,6.7
4,10005,16.211251,5.774971,85.056784,49.311137,12.1,1,0.0,10.7,6.0


In [604]:
df.to_csv('dataAll.csv')

## Read in the Zipcodes shp

In [605]:
zipshp = gpd.read_file('cb_2017_us_zcta510_500k/cb_2017_us_zcta510_500k.shp')
zipshp.shape

(33144, 6)

In [606]:
len(zipshp.ZCTA5CE10.unique())

33144

In [607]:
zipshp.head(3)

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry
0,35442,8600000US35442,35442,610213891,10838694,"(POLYGON ((-88.252618 32.92675, -88.249724 32...."
1,85365,8600000US85365,85365,3545016067,9766486,"(POLYGON ((-114.684663 32.687389, -114.676063 ..."
2,71973,8600000US71973,71973,204670474,1264709,"POLYGON ((-94.46643176650841 34.330735, -94.46..."


## Merging data to shp

In [608]:
dfshp = zipshp.merge(df, left_on = 'ZCTA5CE10', right_on = 'zipcode')
dfshp = dfshp.set_index('zipcode')
dfshp = dfshp.drop(['ZCTA5CE10', 'GEOID10', 'AFFGEOID10', 'ALAND10', 'AWATER10'], axis=1)
print(dfshp.shape)
print(dfshp.columns)

dfshp.head(3)

(1454, 10)
Index(['geometry', '%Foreign', '%Uninsured', '%vehicle', '%foodStamp',
       '%poverty', 'urban', '%TeenBirthRate', '%PrematureBirth',
       '%LowBirthWeight'],
      dtype='object')


,geometry,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%TeenBirthRate,%PrematureBirth,%LowBirthWeight
zipcode,,,,,,,,,,
12832,"POLYGON ((-73.414208 43.388595, -73.4105869999...",0.647249,8.223007,1.463581,40.232421,14.9,1,32.9,7.5,3.7
13421,"POLYGON ((-75.709535 43.088988, -75.704686 43....",1.448316,5.763858,4.992306,39.111895,16.2,1,30.1,8.8,6.7
10456,"POLYGON ((-73.91902899999999 40.83309, -73.914...",20.066442,11.600440,60.622253,32.111269,40.8,1,41.3,12.4,10.2


In [609]:
len(dfshp.index.unique())

1454

In [610]:
dfshp.to_csv('dfshp.csv')

----

# Analysis

# 1. Random Forest

### Split into training and test data

In [611]:
df = df.set_index('zipcode')
df.head()

,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%TeenBirthRate,%PrematureBirth,%LowBirthWeight
zipcode,,,,,,,,,
10001,16.230927,6.313218,77.510070,51.290074,17.5,1,5.7,10.5,11.0
10002,16.301731,7.873643,76.876850,42.674401,27.8,1,17.7,8.9,6.6
10003,12.463227,5.066324,77.210118,48.017334,9.9,1,0.6,9.7,8.7
10004,15.177398,6.241787,76.513672,52.299606,4.8,1,0.0,8.7,6.7
10005,16.211251,5.774971,85.056784,49.311137,12.1,1,0.0,10.7,6.0


In [612]:
df.shape

(1454, 9)

In [613]:
df['%LowBirthWeight'].describe()

count    1454.000000
mean        7.149931
std         3.696764
min         0.000000
25%         5.100000
50%         7.100000
75%         8.900000
max        50.000000
Name: %LowBirthWeight, dtype: float64

In [614]:
df.loc[df['%LowBirthWeight'] <= 5.1] = 1
df.loc[(df['%LowBirthWeight'] > 5.1) & (df['%LowBirthWeight'] <= 7.1), '%LowBirthWeight'] = 2
df.loc[(df['%LowBirthWeight'] > 7.1) & (df['%LowBirthWeight'] <= 8.9), '%LowBirthWeight'] = 3
df.loc[df['%LowBirthWeight'] > 8.9, '%LowBirthWeight'] = 4

df['%LowBirthWeight'].value_counts()

2.0    382
1.0    370
4.0    354
3.0    348
Name: %LowBirthWeight, dtype: int64

In [615]:
df['%LowBirthWeight'][::100]

zipcode
10001    4.0
10532    1.0
10998    4.0
11434    4.0
11778    2.0
12090    4.0
12456    2.0
12760    2.0
12981    1.0
13214    3.0
13607    4.0
13830    1.0
14201    4.0
14559    2.0
14818    4.0
Name: %LowBirthWeight, dtype: float64

In [616]:
df.dropna(0, inplace=True)
df.head(3)

,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%TeenBirthRate,%PrematureBirth,%LowBirthWeight
zipcode,,,,,,,,,
10001,16.230927,6.313218,77.510070,51.290074,17.5,1,5.7,10.5,4.0
10002,16.301731,7.873643,76.876850,42.674401,27.8,1,17.7,8.9,2.0
10003,12.463227,5.066324,77.210118,48.017334,9.9,1,0.6,9.7,3.0


In [617]:
df.dtypes

%Foreign           float64
%Uninsured         float64
%vehicle           float64
%foodStamp         float64
%poverty           float64
urban                int64
%TeenBirthRate     float64
%PrematureBirth    float64
%LowBirthWeight    float64
dtype: object

# 1.1 dependent: _Low birth weight_

In [618]:
X = df.iloc[:,:7]
y = df['%LowBirthWeight']

# X = X.astype(int)
# Y = Y.astype(int)

In [619]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=123)

In [620]:
rf = RandomForestClassifier(n_estimators=30, n_jobs=-1,max_leaf_nodes=10)

In [621]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=10,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [622]:
y_train.dtypes

dtype('float64')

In [623]:
# y_train = np.asarray(y_train, dtype="|S6")

In [624]:
X_train.shape

(955, 7)

In [625]:
y_train.shape

(955,)

In [626]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=10,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [627]:
y_test.dtypes

dtype('float64')

In [628]:
y_test.dtypes

dtype('float64')

In [630]:
max_depth = 20
IS = []
OS = []

for i in range(2, max_depth+1):
    # Split data into 70% train, 30% test
#     X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=999)
    
    # learn model
    rf = RandomForestClassifier(n_estimators=30, n_jobs=-1,max_depth=i)
    rf.fit(X_train,y_train)
    
    # appending to lists >> will be used in 3(b)
    IS.append(rf.score(X_train,y_train)*100)
    OS.append(rf.score(X_test,y_test)*100)
    
    print('max depth = ', i)
    # in sample accuracy
    print('In sample accuracy: {:.4}%'.format(rf.score(X_train,y_train)*100))
    # out of sample accuracy
    print('Out of sample accuracy: {:.4}%'.format(rf.score(X_test,y_test)*100),'\n\n')

max depth =  2
In sample accuracy: 61.05%
Out of sample accuracy: 59.27% 


max depth =  3
In sample accuracy: 62.3%
Out of sample accuracy: 58.05% 


max depth =  4
In sample accuracy: 68.69%
Out of sample accuracy: 61.46% 


max depth =  5
In sample accuracy: 73.61%
Out of sample accuracy: 60.49% 


max depth =  6
In sample accuracy: 76.54%
Out of sample accuracy: 58.29% 


max depth =  7
In sample accuracy: 83.98%
Out of sample accuracy: 60.49% 


max depth =  8
In sample accuracy: 89.95%
Out of sample accuracy: 63.9% 


max depth =  9
In sample accuracy: 91.41%
Out of sample accuracy: 59.76% 


max depth =  10
In sample accuracy: 94.66%
Out of sample accuracy: 58.78% 


max depth =  11
In sample accuracy: 98.01%
Out of sample accuracy: 62.68% 


max depth =  12
In sample accuracy: 99.06%
Out of sample accuracy: 59.76% 


max depth =  13
In sample accuracy: 98.64%
Out of sample accuracy: 59.76% 


max depth =  14
In sample accuracy: 99.69%
Out of sample accuracy: 58.05% 


max depth

In [631]:
np.max(OS)

63.902439024390247

In [632]:
FeatureImportanceLBW = pd.DataFrame(rf.feature_importances_, index=df.iloc[:,:7].columns).T

FeatureImportanceLBW

,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%TeenBirthRate
0,0.114901,0.19955,0.11475,0.249985,0.181518,0.013323,0.125973


## 1.2 dependent: _Preterm Birth_

In [634]:
df['%PrematureBirth'].describe()

count    1365.000000
mean        8.502637
std         5.398773
min         0.000000
25%         1.000000
50%         9.600000
75%        12.300000
max        33.300000
Name: %PrematureBirth, dtype: float64

In [635]:
df.loc[df['%PrematureBirth'] <= 1] = 1
df.loc[(df['%PrematureBirth'] > 1) & (df['%PrematureBirth'] <= 9.6), '%PrematureBirth'] = 2
df.loc[(df['%PrematureBirth'] > 9.6) & (df['%PrematureBirth'] <= 12.3), '%PrematureBirth'] = 3
df.loc[df['%PrematureBirth'] > 12.3, '%PrematureBirth'] = 4

df['%PrematureBirth'].value_counts()

1.0    373
3.0    355
4.0    326
2.0    311
Name: %PrematureBirth, dtype: int64

In [636]:
X = df.iloc[:,:7]
Y = df['%PrematureBirth']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,
                                                   random_state=123)

In [637]:
rf2 = RandomForestClassifier(n_estimators=30, n_jobs=-1,max_leaf_nodes=10)
rf2

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=10,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [638]:
rf2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=10,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [642]:
max_depth = 20
IS = []
OS = []

for i in range(2, max_depth+1):
    # Split data into 70% train, 30% test
#     X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=999)
    
    # learn model
    rf2 = RandomForestClassifier(n_estimators=30, n_jobs=-1,max_depth=i)
    rf2.fit(X_train,y_train)
    
    # appending to lists >> will be used in 3(b)
    IS.append(rf2.score(X_train,y_train)*100)
    OS.append(rf2.score(X_test,y_test)*100)
    
    print('max depth = ', i)
    # in sample accuracy
    print('In sample accuracy: {:.4}%'.format(rf2.score(X_train,y_train)*100))
    # out of sample accuracy
    print('Out of sample accuracy: {:.4}%'.format(rf2.score(X_test,y_test)*100),'\n\n')

max depth =  2
In sample accuracy: 60.0%
Out of sample accuracy: 60.73% 


max depth =  3
In sample accuracy: 64.71%
Out of sample accuracy: 58.54% 


max depth =  4
In sample accuracy: 67.96%
Out of sample accuracy: 59.27% 


max depth =  5
In sample accuracy: 72.36%
Out of sample accuracy: 57.32% 


max depth =  6
In sample accuracy: 76.75%
Out of sample accuracy: 58.78% 


max depth =  7
In sample accuracy: 81.15%
Out of sample accuracy: 58.54% 


max depth =  8
In sample accuracy: 87.23%
Out of sample accuracy: 60.49% 


max depth =  9
In sample accuracy: 92.57%
Out of sample accuracy: 57.32% 


max depth =  10
In sample accuracy: 96.02%
Out of sample accuracy: 56.83% 


max depth =  11
In sample accuracy: 96.34%
Out of sample accuracy: 58.78% 


max depth =  12
In sample accuracy: 98.95%
Out of sample accuracy: 57.32% 


max depth =  13
In sample accuracy: 99.37%
Out of sample accuracy: 58.05% 


max depth =  14
In sample accuracy: 99.58%
Out of sample accuracy: 58.05% 


max dept

In [643]:
np.max(OS)

60.731707317073166

In [645]:
FeatureImportancePMB = pd.DataFrame(rf2.feature_importances_, index=df.iloc[:,:7].columns).T

FeatureImportancePMB

,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%TeenBirthRate
0,0.104882,0.184619,0.138065,0.250522,0.134889,0.015932,0.171092
